# 0. 환경설정

In [4]:
import os
import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup as bts
import json
import re
import time
from tqdm.notebook import tqdm

from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
df22 = pd.read_csv('data/2022_log.csv')
df23 = pd.read_csv('data/2023_log.csv')

In [6]:
df22.head()

,Press,Headline,Analysis
0,IT동아,"잇따르는 전기차 화재, 왜 발생할까?",부정
1,경향신문,"고성능은 기본…전기차, 이제 사이즈로 승부한다",중립
2,오마이뉴스,"""가장 전기차다운…"" 연비-주행 모든 것 담았다",긍정
3,MBC,"싼타페 너마저..전기차 시대, 베트남 중국 인도가 몰려온다",중립
4,오토타임즈,"[시승]엔트리 고급 전기차의 가치, 아우디 Q4 e-트론 40",중립


In [8]:
df22['Headline'] = df22['Headline'].str.replace(pat = '<.+?>', repl = '', regex = True)
df23['Headline'] = df23['Headline'].str.replace(pat = '<.+?>', repl = '', regex = True)

In [9]:
df22['Headline'].head()

0                  잇따르는 전기차 화재, 왜 발생할까?
1             고성능은 기본…전기차, 이제 사이즈로 승부한다
2            "가장 전기차다운…" 연비-주행 모든 것 담았다
3      싼타페 너마저..전기차 시대, 베트남 중국 인도가 몰려온다
4    [시승]엔트리 고급 전기차의 가치, 아우디 Q4 e-트론 40
Name: Headline, dtype: object

In [10]:
os.getcwd()

'C:\\Users\\kim jeongah\\OneDrive\\Desktop\\DMF\\DBR_Project\\DBR_Project\\code'

In [11]:
pd.to_pickle(obj = df22, filepath_or_buffer = 'df22.pkl')
pd.to_pickle(obj = df23, filepath_or_buffer = 'df23.pkl')

In [12]:
os.listdir()

['.DS_Store',
 '.gitignore',
 '.ipynb_checkpoints',
 '00_Library_Installation.ipynb',
 '22시각화-Copy1.ipynb',
 '22시각화.ipynb',
 '22형태소분석-Copy1.ipynb',
 '22형태소분석.ipynb',
 'A1_Naver_News_Link_Func.ipynb',
 'A2_Naver_News_Body_Func.ipynb',
 'A3_Naver_News_Reply_Func.ipynb',
 'B1_Text_Mining.ipynb',
 'B2_Visualization.ipynb',
 'B3_Topic_Modeling.ipynb',
 'data',
 'df22.pkl',
 'df23.pkl',
 'venv']

In [13]:
os.chdir(path = '../data')

In [14]:
sorted(os.listdir())

['22_Prep.pkl',
 '23_Prep.pkl',
 'Naver_News_List.pkl',
 'Naver_News_Reply.pkl',
 'Text_Prep.pkl',
 'df22.pkl',
 'df23.pkl']

In [15]:
df22 = pd.read_pickle(filepath_or_buffer = 'df22.pkl')
df23 = pd.read_pickle(filepath_or_buffer = 'df23.pkl')

In [103]:
df23.head()

,Press,Headline,Analysis
0,연합뉴스,"'내년 첫 전기차 출시' 中샤오미 ""테슬라 따라잡을 준비됐다""",긍정
1,YTN,"'자국 우선주의' 허들 만난 '전기차', 묘수는?",중립
2,연합뉴스,"애플 따라하던 中 샤오미, 전기차는 먼저 치고나가",긍정
3,연합뉴스,"美 전기차 전환, 정부 지원에도 수요 감소로 기대보다 느려",부정
4,매일경제,주춤한 전기차 … 내년 보급형 몰려온다,중립


In [104]:
df22.head()

,Press,Headline,Analysis
0,IT동아,"잇따르는 전기차 화재, 왜 발생할까?",부정
1,경향신문,"고성능은 기본…전기차, 이제 사이즈로 승부한다",중립
2,오마이뉴스,"""가장 전기차다운…"" 연비-주행 모든 것 담았다",긍정
3,MBC,"싼타페 너마저..전기차 시대, 베트남 중국 인도가 몰려온다",중립
4,오토타임즈,"[시승]엔트리 고급 전기차의 가치, 아우디 Q4 e-트론 40",중립


In [16]:
df22.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 782 entries, 0 to 781
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Press     782 non-null    object
 1   Headline  782 non-null    object
 2   Analysis  782 non-null    object
dtypes: object(3)
memory usage: 18.5+ KB


In [17]:
sens = df22['Headline']

In [18]:
sens.head(100)

0                     잇따르는 전기차 화재, 왜 발생할까?
1                고성능은 기본…전기차, 이제 사이즈로 승부한다
2               "가장 전기차다운…" 연비-주행 모든 것 담았다
3         싼타페 너마저..전기차 시대, 베트남 중국 인도가 몰려온다
4       [시승]엔트리 고급 전기차의 가치, 아우디 Q4 e-트론 40
                      ...                 
95                전기차, '인포테인먼트' 품고 뉴노멀車 될까
96     "내년엔 보조금 폐지, 빨리 사자"…中, 전기차 불티나게 팔렸다
97                 떠오르는 동남아 전기차 시장 한중일 각축전
98    [시승기] 맘놓고 오래 달려도 걱정없는 전기차..현대차 아이오닉6
99                  [시론] 빠르게 다가오는 '전기차 혁명'
Name: Headline, Length: 100, dtype: object

In [19]:
 i=0
sens.iloc[1]

'고성능은 기본…전기차, 이제 사이즈로 승부한다'

# 0.1 빈도수 정확도를 높이기 위한 튜닝

In [69]:
# sens에서 지정한 패턴이 아닌 글자를 공백으로 변경하고 sens에 재할당
sens = sens.str.replace(pat = '[^가-힣A-Za-z0-9.]|&.+;', repl = '', regex = True)

# 일부 단어를 변경함
sens = sens.str.replace(pat = 'e트론', repl = '아우디', regex = True)
# sens = sens.str.replace(pat = '전기', repl = '전기차', regex = True)
sens = sens.str.replace(pat = '전기', repl = '', regex = True)
sens = sens.str.replace(pat = '기차', repl = '', regex = True)
sens = sens.str.replace(pat = '차다', repl = '', regex = True)

# 1. 형태소 분석

In [70]:
kiwistop = Stopwords()

In [71]:
kiwi = Kiwi()

In [72]:
sens[i]

'시승엔트리고급차의가치아우디Q4아우디40'

In [73]:
kiwi.tokenize(text = sens[i], stopwords = kiwistop)

[Token(form='시승', tag='NNG', start=0, len=2),
 Token(form='엔트리', tag='NNG', start=2, len=3),
 Token(form='고급차', tag='NNG', start=5, len=3),
 Token(form='가치', tag='NNG', start=9, len=2),
 Token(form='아우디', tag='NNG', start=11, len=3),
 Token(form='Q', tag='SL', start=14, len=1),
 Token(form='4', tag='SN', start=15, len=1),
 Token(form='아우디', tag='NNG', start=16, len=3),
 Token(form='40', tag='SN', start=19, len=2)]

In [74]:
words = ['인도','e트론']

In [75]:
for word in words:
    kiwi.add_user_word(word = word, tag = 'NNP', score = 15)

In [76]:
tokens = kiwi.tokenize(text = sens[i], stopwords = kiwistop)

tokens

[Token(form='시승', tag='NNG', start=0, len=2),
 Token(form='엔트리', tag='NNG', start=2, len=3),
 Token(form='고급차', tag='NNG', start=5, len=3),
 Token(form='가치', tag='NNG', start=9, len=2),
 Token(form='아우디', tag='NNG', start=11, len=3),
 Token(form='Q', tag='SL', start=14, len=1),
 Token(form='4', tag='SN', start=15, len=1),
 Token(form='아우디', tag='NNG', start=16, len=3),
 Token(form='40', tag='SN', start=19, len=2)]

In [77]:
print(f'형태:{tokens[5].form}')
print(f'품사:{tokens[5].tag}')

형태:Q
품사:SL


In [78]:
pos1, pos2 = ['VV', 'VA'], ['NNG', 'NNP']

In [79]:
# 형태소의 품사가 용언과 체언인 형태소만 선택하고 품사가 용언인 형태소에 
# 종결어미 '다'를 결합하여 리스트로 반환

tokens = [token.form + '다' if token.tag in pos1 else token.form 
          for token in tokens if token.tag in pos1 + pos2]

tokens

['시승', '엔트리', '고급차', '가치', '아우디', '아우디']

# 2. 말뭉치 생성

In [80]:
# 말뭉치를 저장할 빈 리스트 생성
corpus = list()

# sens의 원소(문서)를 형태소 분석하고 일부 품사를 선택하여 말뭉치에 추가
for sen in sens:
    tokens = kiwi.tokenize(text = sen, stopwords = kiwistop)
    tokens = [token.form + '다' if token.tag in pos1 else token.form 
              for token in tokens if token.tag in pos1 + pos2]
    corpus.append(tokens)

In [81]:
# 형태소 분석 결과(corpus)의 처음 5개 원소 확인
# [참고] 원본 문서와 결과를 비교하여 사용자 사전에 추가하는 것을 반복해야 함
for i in range(5):
    print(corpus[i])

['잇따르다', '차', '화재', '발생']
['성능', '기본', '사이즈', '승부']
['운', '연비', '주행', '담다']
['싼타페', '차시', '베트남', '중국', '인도', '몰려오다']
['시승', '엔트리', '고급차', '가치', '아우디', '아우디']


In [82]:
for i in range(5):
    print(df22['Headline'].iloc[i], '\n')

잇따르는 전기차 화재, 왜 발생할까? 

고성능은 기본…전기차, 이제 사이즈로 승부한다 

"가장 전기차다운…" 연비-주행 모든 것 담았다 

싼타페 너마저..전기차 시대, 베트남 중국 인도가 몰려온다 

[시승]엔트리 고급 전기차의 가치, 아우디 Q4 e-트론 40 



# 3. 문서-단어 행렬 생성

In [83]:
# corpus의 각 원소(리스트)를 하나의 문자열로 결합하여 corpus에 재할당
# [참고] 문서-단어 행렬 생성하려면 corpus 원소가 문자열이어야 함
corpus = [' '.join(i) for i in corpus]

# corpus의 처음 5개 확인
corpus[0:5]

['잇따르다 차 화재 발생',
 '성능 기본 사이즈 승부',
 '운 연비 주행 담다',
 '싼타페 차시 베트남 중국 인도 몰려오다',
 '시승 엔트리 고급차 가치 아우디 아우디']

In [84]:
# 문서별 단어의 tf를 계산하는 객체 생성
# [참고] TfidfVectorizer()는 단어의 tf-idf를 반환
cv = CountVectorizer()

In [85]:
# 단어의 tf를 성분으로 갖는 문서-단어 행렬 생성
dtm = cv.fit_transform(raw_documents = corpus).toarray()

# dtm 확인
# [참고] dtm은 2차원 행렬임
dtm

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [86]:
# cv 객체에 저장된 단어 목록 확인
# [참고] 알파벳이 모두 소문자로 바뀌었음
cv.get_feature_names_out()

array(['1위비야디', 'gm포드차', 'k차', ..., '흔들리다', '희비', '히터'], dtype=object)

In [87]:
# dtm을 데이터프레임으로 변환
dtm = pd.DataFrame(data = dtm, columns = cv.get_feature_names_out())

In [88]:
# dtm의 열이름에서 알파벳을 대문자로 변경
dtm.columns = dtm.columns.str.upper()

In [89]:
dtm.head()

,1위비야디,GM포드차,K차,LG유플러스,가격,가능,가다,가동,가로등,가로막다,...,휘발유,휠체어,휴가철,휴게소,휴일,흉내,흑자,흔들리다,희비,히터
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [90]:
dtm.shape

(782, 1705)

In [91]:
tfs = dtm.sum().sort_values(ascending = False)

In [92]:
# tfs의 상위 10개 및 하위 10개 확인
display(tfs.head(n = 10))
display(tfs.tail(n = 10))

보조금    108
한국      59
배터리     56
충전      51
테슬라     47
시장      45
중국      43
미국      35
차별      33
차다      31
dtype: int64

사이즈     1
사각지대    1
사운드     1
사용자     1
사수      1
사상      1
사물      1
사무소     1
사례      1
히터      1
dtype: int64

In [93]:
# tfs가 20 이하인 인덱스(단어)를 dtm에서 제거
# [참고] 단어 빈도수를 높일수록 dtm의 열(차원) 개수가 감소함
threshold = 20
dtm = dtm.drop(columns = tfs.loc[tfs.le(threshold)].index)

In [94]:
# dtm의 행 개수와 열 개수 확인
# [참고] 단어 빈도수가 매우 작은 일부 단어를 삭제함으로써 차원(열) 축소 가능
# [참고] dtm의 차원을 축소하면 행렬곱 연산을 빠르게 실행할 수 있음
dtm.shape

(782, 20)

In [95]:
# 현재 작업 경로 확인
os.getcwd()

'C:\\Users\\kim jeongah\\OneDrive\\Desktop\\DMF\\DBR_Project\\DBR_Project\\data'

In [96]:
# corpus, tfs 및 dtm을 하나의 pkl 파일로 저장
pd.to_pickle(obj = [corpus, tfs, dtm], filepath_or_buffer = '22_Prep.pkl')

In [149]:
# 현재 작업 경로에 있는 폴더명과 파일명 확인
os.listdir()

['22_Prep.pkl',
 'df22.pkl',
 'df23.pkl',
 'Naver_News_List.pkl',
 'Naver_News_Reply.pkl',
 'Text_Prep.pkl']

In [97]:
df22.head()

,Press,Headline,Analysis
0,IT동아,"잇따르는 전기차 화재, 왜 발생할까?",부정
1,경향신문,"고성능은 기본…전기차, 이제 사이즈로 승부한다",중립
2,오마이뉴스,"""가장 전기차다운…"" 연비-주행 모든 것 담았다",긍정
3,MBC,"싼타페 너마저..전기차 시대, 베트남 중국 인도가 몰려온다",중립
4,오토타임즈,"[시승]엔트리 고급 전기차의 가치, 아우디 Q4 e-트론 40",중립


In [100]:
df22['Analysis'].value_counts(normalize=True)

Analysis
중립    0.367008
부정    0.356777
긍정    0.276215
Name: proportion, dtype: float64